In [1]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
sys.path.append(os.path.join(os.getcwd(), 'domain'))
from tests.domain.primary.domain.create_domain import DomainTables

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float32'
dim=3
mesh_name = 'tetrahedron.msh'

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'domain', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=4, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

path /media/aben-ham/SSD/aben-ham/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 0
====> Start <=====
====> Creating Mesh <=====

path /media/aben-ham/SSD/aben-ham/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 3
path /media/aben-ham/SSD/aben-ham/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 1
path /media/aben-ham/SSD/aben-ham/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 2
node_cellid
Execution time: 1.669638 seconds
cell_cellnid
Execution time: 2.144686 seconds
cell_cellfid and boundary_faces
Execution time: 4.362438 seconds
node_bfid
Execution time: 4.362970 seconds
Start creating sub domains
Parse Input 
Start Creating SubDomains 
Make n part
Start dense to csr
Nb_vertices = 6000
size = 22800
End dense to csr
End METIS_PartGraphKway
End function
Create local cells
Create Physical faces
Create local domains struct
Create local domains stru

In [3]:
import importlib

module = importlib.import_module("helpers.TablesTestTetra3D")
importlib.reload(module)
TablesTestTetra3D = getattr(module, "TablesTestTetra3D")

module = importlib.import_module("helpers.TetraChecker3D")
importlib.reload(module)
TetraChecker3D = getattr(module, "TetraChecker3D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TablesTestTetra3D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = TetraChecker3D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_node_info()
# checker.test_face_info()
checker.summary()
# checker.test_halo_info()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 13 [P] 1 [F] Total 14
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/14: Cell Vertices
[P] 2/14: Cell Center
[P] 3/14: Cell Area
[P] 4/14: Cell Neighbors by face
[P] 5/14: Cell Neighbors by node
* [F] 6/14: Cell Halo by face 1 / 6000 799 Failed
	
	Arrays are not equal
	
	(shapes (1,), (0,) mismatch)
	 x: array([36], dtype=int32)
	 y: array([], dtype=int32)
[P] 7/14: Cell Halo by node
[P] 8/14: Cell Ghostnid *
[P] 9/14: Cell Haloghostnid and Haloghostcenter *
[P] 10/14: Cell Number of cells
[P] 11/14: Node Cellid
[P] 12/14: Node Loctoglob
[P] 13/14: Node Halonid
[P] 14/14: Node Number of nodes

---------------------------------------------------------
---------------------------------------------------------
===> 13 [P] 1 [F] Total 14


False

In [3]:
halonid = test_tables.cell_haloghostnid[3363]
print(halonid)

[565 568 569 641 644 720 721  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1   7]


In [4]:
print(test_tables.node_haloghostid[3363])

[[641 644 720 721  -1  -1   4]
 [565 568 569 644  -1  -1   4]
 [ -1  -1  -1  -1  -1  -1   0]
 [ -1  -1  -1  -1  -1  -1   0]]


In [6]:
p = 2
i = 10

for i in range(domain_tables.d_cell_haloghostnid[p].shape[0]):
  c_haloghostnid = domain_tables.d_cell_haloghostnid[p][i]
  c_haloghostnid = c_haloghostnid[0:c_haloghostnid[-1]]
  c_haloghostcenter = domain_tables.d_cell_haloghostcenter[p][c_haloghostnid][:, 0:3] # center (x, y, z)

  if (c_haloghostnid.shape[0] != 0 and i == 10):
    print(c_haloghostcenter)


In [7]:
p = 2
i = 10

cell = domain_tables.d_cells[p][i]
nodes = cell[0:cell[-1]]
print(nodes)
print(domain_tables.d_node_halobfid[p][nodes])

for k in range(domain_tables.d_bcell_halobfid[p].shape[0]):
  if domain_tables.d_bcell_halobfid[p][k, 0] == i:
    print(domain_tables.d_bcell_halobfid[p][k])
print(domain_tables.d_cell_haloghostnid[p][i])

[12 13 14 15]
[[ 24  23  32  30   0   4]
 [ 38  39 407  23   0   4]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]
[10  0  0  0  0  0  0  0  0  1]
[0 0 0 0 0 0 0 0 0]


In [14]:
c_haloghostnid = domain_tables.d_cell_haloghostnid[p][i]
c_haloghostnid = c_haloghostnid[0:c_haloghostnid[-1]]
print(domain_tables.d_cell_haloghostcenter[p].shape)

(93, 3)


In [6]:
print(domain_tables.d_cell_ghostnid[0][0:10])

[[  2 441   5  65  73  15  16  19 444 456   0   0   0   0   0   0   0  10]
 [  2   5  15  16  19 441 444 456  65  73  22  78  87   0   0   0   0  13]
 [  2 441   5  65  73  15  19 444 456   0   0   0   0   0   0   0   0   9]
 [  5  65  73  16  22  78  87   2  15  19 441 444 456   0   0   0   0  13]
 [  2  15  19 441 444 456   5  16  22  73  78  87   0   0   0   0   0  12]
 [  2   5  15  16  19 441 444 456  22  73  78  87  31  32   0   0   0  14]
 [  2  15  19 441 444 456   5  16  22  73  78  87  31  32  35 459 470  17]
 [ 15  16  19  22  31  32  35 456 459 470   5  73  78  87  38  92 101  17]
 [  2  15  19 441 444 456   5  16  22  73  78  87  31  35 459 470   0  16]
 [  5  16  22  73  78  87  32  38  92 101  19  31  35 456 459 470   0  16]]


In [118]:
print(domain_tables.d_cell_ghostnid[p].shape)

(1500, 25)


In [48]:
print(domain_tables.d_node_halonid[0])

[[  0   1   2 ...   0   0   9]
 [  1   3   4 ...   0   0   6]
 [  5   0   0 ...   0   0   1]
 ...
 [784 824 785 ...   0   0  10]
 [793 795 796 ...   0   0   9]
 [807 801 802 ...   0   0   9]]


In [68]:
def _is_in_array(array: 'int[:]', item: 'int') -> 'int':
  for i in range(array[-1]):
    if item == array[i]:
      return 1
  return 0

p = 0
cells = domain_tables.d_cells[p]
nb_cells = len(cells)
node_haloid = domain_tables.d_node_halonid[p]

cell_halonid = np.zeros(shape=(nb_cells, 20 + 1), dtype=np.int32)
for i in range(nb_cells):
  print(cells[i])
  for j in range(cells[i, -1]):
    n_halo = node_haloid[cells[i, j]]
    print("=>", n_halo)
    for k in range(n_halo[-1]):
      if _is_in_array(cell_halonid[i], n_halo[k]) == 0:
        size = cell_halonid[i, -1]
        cell_halonid[i, -1] += 1
        cell_halonid[i, size] = n_halo[k]
  print("Res: ", cell_halonid[i])
  if i == 2:
      break

# print(cell_halonid[0:10])

[0 1 2 3 4]
=> [ 0  1  2  3  4  5  8  9 10  0  0  0  0  0  0  0  0  0  0  0  0  0  9]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Res:  [ 0  1  2  3  4  5  8  9 10  6  7  0  0  0  0  0  0  0  0  0 11]
[2 3 1 4 4]
=> [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Res:  [5 1 3 4 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[0 1 3 5 4]
=> [ 0  1  2  3  4  5  8  9 10  0  0  0  0  0  0  0  0  0  0  0  0  0  9]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
=> [ 4  7 11 10 12 13 14 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8]
Res:  [ 0  1  2  3  4  5  8  9 10  6  7 11 12 13 14 15  0  0  0  0 16]


In [60]:
d_nodes = domain_tables.d_nodes[p]
d_cells = domain_tables.d_cells[p]
i = 400
c_cell_vertices = d_nodes[d_cells[i][0:d_cells[i][-1]]]
print(c_cell_vertices)

[[2.  1.  9. ]
 [3.  1.  9. ]
 [2.  1.5 7.5]
 [2.  1.5 9. ]]
